In [30]:
import pandas as pd
import copy
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout

In [2]:
pokemon_151 = pd.read_csv('Pokemon_151.csv', index_col = 'Unnamed: 0')
pokemon_151

,Product,Foil,Date Range,Market Price
0,151: Alakazam ex Collection,[' Market Price: '],8/5 to 8/7,$27.46
1,151: Alakazam ex Collection,[' Market Price: '],8/8 to 8/10,$26.81
2,151: Alakazam ex Collection,[' Market Price: '],8/11 to 8/13,$26.21
3,151: Alakazam ex Collection,[' Market Price: '],8/14 to 8/16,$25.41
4,151: Alakazam ex Collection,[' Market Price: '],8/17 to 8/19,$23.35
...,...,...,...,...
11785,Zubat,['Reverse Holofoil Market Price: '],10/25 to 10/27,$0.20
11786,Zubat,[' Market Price: '],10/28 to 10/30,$0.04
11787,Zubat,['Reverse Holofoil Market Price: '],10/28 to 10/30,$0.23
11788,Zubat,[' Market Price: '],10/31 to 11/2,$0.03


In [3]:
len(pokemon_151['Product'].unique())

240

In [4]:
pokemon_151['Foil'].unique()

array(["[' Market Price: ']", "['Reverse Holofoil Market Price: ']",
       "['Holofoil Market Price: ']"], dtype=object)

In [5]:
pokemon_151.replace({'Foil' : { "[' Market Price: ']" : "Normal", "[' Foil Market Price: ']" : "Foil",
                               "['Reverse Holofoil Market Price: ']" : "Reverse Holofoil", 
                               "['Holofoil Market Price: ']" : "Holofoil"}}, inplace=True)

In [6]:
pokemon_151

,Product,Foil,Date Range,Market Price
0,151: Alakazam ex Collection,Normal,8/5 to 8/7,$27.46
1,151: Alakazam ex Collection,Normal,8/8 to 8/10,$26.81
2,151: Alakazam ex Collection,Normal,8/11 to 8/13,$26.21
3,151: Alakazam ex Collection,Normal,8/14 to 8/16,$25.41
4,151: Alakazam ex Collection,Normal,8/17 to 8/19,$23.35
...,...,...,...,...
11785,Zubat,Reverse Holofoil,10/25 to 10/27,$0.20
11786,Zubat,Normal,10/28 to 10/30,$0.04
11787,Zubat,Reverse Holofoil,10/28 to 10/30,$0.23
11788,Zubat,Normal,10/31 to 11/2,$0.03


In [7]:
sum(pokemon_151['Market Price'] == "$0.00") / len(pokemon_151['Market Price'] == "$0.00")
# 49% of dataset is $0.00

0.49236641221374045

In [8]:
pokemon_151 = pokemon_151[pokemon_151['Market Price'] != '$0.00']
pokemon_151['Market Price'] = pokemon_151['Market Price'].replace({'\$':''}, regex = True)
pokemon_151['Market Price'] = pokemon_151['Market Price'].astype(float)
pokemon_151 = pokemon_151.reset_index(drop=True)
pokemon_151

C:\Users\steve\AppData\Local\Temp\ipykernel_29012\706122419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pokemon_151['Market Price'] = pokemon_151['Market Price'].replace({'\$':''}, regex = True)
C:\Users\steve\AppData\Local\Temp\ipykernel_29012\706122419.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pokemon_151['Market Price'] = pokemon_151['Market Price'].astype(float)


,Product,Foil,Date Range,Market Price
0,151: Alakazam ex Collection,Normal,8/5 to 8/7,27.46
1,151: Alakazam ex Collection,Normal,8/8 to 8/10,26.81
2,151: Alakazam ex Collection,Normal,8/11 to 8/13,26.21
3,151: Alakazam ex Collection,Normal,8/14 to 8/16,25.41
4,151: Alakazam ex Collection,Normal,8/17 to 8/19,23.35
...,...,...,...,...
5980,Zubat,Reverse Holofoil,10/25 to 10/27,0.20
5981,Zubat,Normal,10/28 to 10/30,0.04
5982,Zubat,Reverse Holofoil,10/28 to 10/30,0.23
5983,Zubat,Normal,10/31 to 11/2,0.03


In [9]:
def past_prices(num_historical_prices, df):
    dataset = copy.deepcopy(df)
    for n in range(num_historical_prices):
        # whether product matches the product from the previous line
        match_prev_product = [dataset['Product'][i] == dataset['Product'][i+1] for i in range(len(dataset['Product'])-1)]
        # whether foil matches the foil from the previous line
        match_prev_foil = [dataset['Foil'][i] == dataset['Foil'][i+1] for i in range(len(dataset['Foil'])-1)]
        # base case, take the prev market price from the previous row
        if n == 0:
            prev_market_price = dataset['Market Price'][:-1]
        # otherwise take the latest next market price from the next row
        else:
            prev_market_price = dataset[f'Previous Market Price_{n}'][:-1]
            
        dataset = dataset[1:]
        dataset = dataset.reset_index(drop=True)
        dataset[f'Previous Market Price_{n+1}'] = prev_market_price
        prev_match = [match_prev_product[i] and match_prev_foil[i] for i in range(len(match_prev_product))]
        dataset = dataset[prev_match]
        dataset = dataset.reset_index(drop=True)
    return dataset

In [10]:
def future_prices(num_historical_prices, df):
    dataset = copy.deepcopy(df)
    for n in range(num_historical_prices):
        match_next_product = [dataset['Product'][i] == dataset['Product'][i+1] for i in range(len(dataset['Product'])-1)]
        match_next_foil = [dataset['Foil'][i] == dataset['Foil'][i+1] for i in range(len(dataset['Foil'])-1)]
        # base case, take the next market price from the next row
        if n == 0:
            next_market_price = dataset['Market Price'][1:]
        # otherwise take the latest next market price from the next row
        else:
            next_market_price = dataset[f'Next Market Price_{n}'][1:]
                    
        next_market_price = next_market_price.reset_index(drop=True)
        dataset = dataset[:-1]
        dataset = dataset.reset_index(drop=True)
        dataset[f'Next Market Price_{n+1}'] = next_market_price
        next_match = [match_next_product[i] and match_next_foil[i] for i in range(len(match_next_product))]
        dataset = dataset[next_match]
        dataset = dataset.reset_index(drop=True)
    return dataset

In [11]:
# gets the next month of market prices
pokemon_151_future = future_prices(1, pokemon_151)
# obtains the last 15 days worth of prices
pokemon_151_past_future_prices = past_prices(5, pokemon_151_future)
pokemon_151_past_future_prices

,Product,Foil,Date Range,Market Price,Next Market Price_1,Previous Market Price_1,Previous Market Price_2,Previous Market Price_3,Previous Market Price_4,Previous Market Price_5
0,151: Alakazam ex Collection,Normal,8/20 to 8/22,22.63,22.63,23.35,25.41,26.21,26.81,27.46
1,151: Alakazam ex Collection,Normal,8/23 to 8/25,22.63,22.62,22.63,23.35,25.41,26.21,26.81
2,151: Alakazam ex Collection,Normal,8/26 to 8/28,22.62,22.50,22.63,22.63,23.35,25.41,26.21
3,151: Alakazam ex Collection,Normal,8/29 to 8/31,22.50,22.42,22.62,22.63,22.63,23.35,25.41
4,151: Alakazam ex Collection,Normal,9/1 to 9/3,22.42,22.36,22.50,22.62,22.63,22.63,23.35
...,...,...,...,...,...,...,...,...,...,...
853,Zapdos ex - 202/165,Holofoil,10/16 to 10/18,42.84,41.67,43.75,46.11,43.29,39.52,43.81
854,Zapdos ex - 202/165,Holofoil,10/19 to 10/21,41.67,42.31,42.84,43.75,46.11,43.29,39.52
855,Zapdos ex - 202/165,Holofoil,10/22 to 10/24,42.31,42.06,41.67,42.84,43.75,46.11,43.29
856,Zapdos ex - 202/165,Holofoil,10/25 to 10/27,42.06,41.97,42.31,41.67,42.84,43.75,46.11


# Data Split

In [13]:
pokemon_151_past_future_prices['Date Range'].unique()

array(['8/20 to 8/22', '8/23 to 8/25', '8/26 to 8/28', '8/29 to 8/31',
       '9/1 to 9/3', '9/4 to 9/6', '9/7 to 9/9', '9/10 to 9/12',
       '9/13 to 9/15', '9/16 to 9/18', '9/19 to 9/21', '9/22 to 9/24',
       '9/25 to 9/27', '9/28 to 9/30', '10/1 to 10/3', '10/4 to 10/6',
       '10/7 to 10/9', '10/10 to 10/12', '10/13 to 10/15',
       '10/16 to 10/18', '10/19 to 10/21', '10/22 to 10/24',
       '10/25 to 10/27', '10/28 to 10/30'], dtype=object)

In [14]:
test = pokemon_151_past_future_prices.loc[pokemon_151_past_future_prices['Date Range'].isin(['10/16 to 10/18', '10/19 to 10/21', '10/22 to 10/24',
       '10/25 to 10/27', '10/28 to 10/30'])]
train = pokemon_151_past_future_prices.loc[pokemon_151_past_future_prices['Date Range'].isin(['8/20 to 8/22', '8/23 to 8/25', '8/26 to 8/28', '8/29 to 8/31',
       '9/1 to 9/3', '9/4 to 9/6', '9/7 to 9/9', '9/10 to 9/12',
       '9/13 to 9/15', '9/16 to 9/18', '9/19 to 9/21', '9/22 to 9/24',
       '9/25 to 9/27', '9/28 to 9/30', '10/1 to 10/3', '10/4 to 10/6',
       '10/7 to 10/9', '10/10 to 10/12', '10/13 to 10/15'])]
test

,Product,Foil,Date Range,Market Price,Next Market Price_1,Previous Market Price_1,Previous Market Price_2,Previous Market Price_3,Previous Market Price_4,Previous Market Price_5
19,151: Alakazam ex Collection,Normal,10/16 to 10/18,24.40,25.65,22.82,25.33,24.35,26.56,31.60
20,151: Alakazam ex Collection,Normal,10/19 to 10/21,25.65,26.64,24.40,22.82,25.33,24.35,26.56
21,151: Alakazam ex Collection,Normal,10/22 to 10/24,26.64,25.79,25.65,24.40,22.82,25.33,24.35
22,151: Alakazam ex Collection,Normal,10/25 to 10/27,25.79,25.26,26.64,25.65,24.40,22.82,25.33
23,151: Alakazam ex Collection,Normal,10/28 to 10/30,25.26,25.40,25.79,26.64,25.65,24.40,22.82
...,...,...,...,...,...,...,...,...,...,...
853,Zapdos ex - 202/165,Holofoil,10/16 to 10/18,42.84,41.67,43.75,46.11,43.29,39.52,43.81
854,Zapdos ex - 202/165,Holofoil,10/19 to 10/21,41.67,42.31,42.84,43.75,46.11,43.29,39.52
855,Zapdos ex - 202/165,Holofoil,10/22 to 10/24,42.31,42.06,41.67,42.84,43.75,46.11,43.29
856,Zapdos ex - 202/165,Holofoil,10/25 to 10/27,42.06,41.97,42.31,41.67,42.84,43.75,46.11


In [15]:
X_test = test[[ 'Foil', 'Previous Market Price_1', 'Previous Market Price_2', 'Previous Market Price_3',
              'Previous Market Price_4', 'Previous Market Price_5', 'Market Price']]
y_test = test['Next Market Price_1'].astype(float)
X_train = train[[ 'Foil', 'Previous Market Price_1', 'Previous Market Price_2', 'Previous Market Price_3',
              'Previous Market Price_4', 'Previous Market Price_5', 'Market Price']]
y_train = train['Next Market Price_1'].astype(float)

In [16]:
X_test = pd.get_dummies(X_test, columns = ['Foil'])
X_train = pd.get_dummies(X_train, columns = ['Foil'])

# Linear Regression

In [19]:
classifier = LinearRegression()
classifier.fit(X_train, y_train)

LinearRegression()

In [20]:
coefficients = pd.DataFrame(zip(X_train.columns, classifier.coef_))
coefficients

,0,1
0,Previous Market Price_1,-0.656283
1,Previous Market Price_2,0.371770
2,Previous Market Price_3,-0.025742
3,Previous Market Price_4,-0.062820
4,Previous Market Price_5,0.010813
5,Market Price,1.354958
6,Foil_Holofoil,0.270795
7,Foil_Normal,-0.270795


In [21]:
y_pred = classifier.predict(X_test)

In [22]:
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
#     mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
#     print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [27]:
regression_results(y_test, y_pred)

explained_variance:  0.9996
r2:  0.9996
MAE:  1.1811
MSE:  3.871
RMSE:  1.9675


# LSTM

In [28]:
X_train = train[['Previous Market Price_5', 'Previous Market Price_4', 'Previous Market Price_3',
              'Previous Market Price_2', 'Previous Market Price_1', 'Market Price']]

X_test = test[['Previous Market Price_5', 'Previous Market Price_4', 'Previous Market Price_3',
              'Previous Market Price_2', 'Previous Market Price_1', 'Market Price']]

In [37]:
model = Sequential()  
model.add(LSTM(2, input_dim=1, return_sequences=False))
# model.add(Dropout(0.5))
# model.add(Dense(8, activation='linear'))
model.add(Dense(1, activation='linear'))  
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mean_squared_error'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 2)                 32        
                                                                 
 dense_4 (Dense)             (None, 1)                 3         
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [38]:
history = model.fit(X_train, y_train,epochs=20000, validation_data=(X_test, y_test))

Epoch 1/20000
16/16 [==============================] - 2s 32ms/step - loss: 23449.6074 - mean_squared_error: 23449.6074 - val_loss: 10742.2217 - val_mean_squared_error: 10742.2217
Epoch 2/20000
16/16 [==============================] - 0s 4ms/step - loss: 23402.2266 - mean_squared_error: 23402.2266 - val_loss: 10736.3594 - val_mean_squared_error: 10736.3594
Epoch 3/20000
16/16 [==============================] - 0s 4ms/step - loss: 23394.2363 - mean_squared_error: 23394.2363 - val_loss: 10731.9951 - val_mean_squared_error: 10731.9951
Epoch 4/20000
16/16 [==============================] - 0s 5ms/step - loss: 23387.5742 - mean_squared_error: 23387.5742 - val_loss: 10727.7676 - val_mean_squared_error: 10727.7676
Epoch 5/20000
16/16 [==============================] - 0s 5ms/step - loss: 23381.1719 - mean_squared_error: 23381.1719 - val_loss: 10723.7725 - val_mean_squared_error: 10723.7725
Epoch 6/20000
16/16 [==============================] - 0s 4ms/step - loss: 23375.0469 - mean_squared_err

16/16 [==============================] - 0s 5ms/step - loss: 22985.5566 - mean_squared_error: 22985.5566 - val_loss: 10502.9844 - val_mean_squared_error: 10502.9844
Epoch 47/20000
16/16 [==============================] - 0s 5ms/step - loss: 22978.2637 - mean_squared_error: 22978.2637 - val_loss: 10499.2939 - val_mean_squared_error: 10499.2939
Epoch 48/20000
16/16 [==============================] - 0s 5ms/step - loss: 22971.6621 - mean_squared_error: 22971.6621 - val_loss: 10495.5586 - val_mean_squared_error: 10495.5586
Epoch 49/20000
16/16 [==============================] - 0s 5ms/step - loss: 22964.9414 - mean_squared_error: 22964.9414 - val_loss: 10491.9502 - val_mean_squared_error: 10491.9502
Epoch 50/20000
16/16 [==============================] - 0s 4ms/step - loss: 22958.2598 - mean_squared_error: 22958.2598 - val_loss: 10488.2910 - val_mean_squared_error: 10488.2910
Epoch 51/20000
16/16 [==============================] - 0s 4ms/step - loss: 22951.6211 - mean_squared_error: 22951.

16/16 [==============================] - 0s 5ms/step - loss: 22690.0781 - mean_squared_error: 22690.0781 - val_loss: 10342.3027 - val_mean_squared_error: 10342.3027
Epoch 92/20000
16/16 [==============================] - 0s 5ms/step - loss: 22683.9453 - mean_squared_error: 22683.9453 - val_loss: 10338.8604 - val_mean_squared_error: 10338.8604
Epoch 93/20000
16/16 [==============================] - 0s 5ms/step - loss: 22677.6113 - mean_squared_error: 22677.6113 - val_loss: 10335.2832 - val_mean_squared_error: 10335.2832
Epoch 94/20000
16/16 [==============================] - 0s 5ms/step - loss: 22670.8457 - mean_squared_error: 22670.8457 - val_loss: 10331.5342 - val_mean_squared_error: 10331.5342
Epoch 95/20000
16/16 [==============================] - 0s 5ms/step - loss: 22664.1113 - mean_squared_error: 22664.1113 - val_loss: 10328.1406 - val_mean_squared_error: 10328.1406
Epoch 96/20000
16/16 [==============================] - 0s 5ms/step - loss: 22657.8359 - mean_squared_error: 22657.

16/16 [==============================] - 0s 5ms/step - loss: 22403.4336 - mean_squared_error: 22403.4336 - val_loss: 10188.9014 - val_mean_squared_error: 10188.9014
Epoch 137/20000
16/16 [==============================] - 0s 5ms/step - loss: 22398.1035 - mean_squared_error: 22398.1035 - val_loss: 10185.6221 - val_mean_squared_error: 10185.6221
Epoch 138/20000
16/16 [==============================] - 0s 5ms/step - loss: 22391.9199 - mean_squared_error: 22391.9199 - val_loss: 10182.5117 - val_mean_squared_error: 10182.5117
Epoch 139/20000
16/16 [==============================] - 0s 5ms/step - loss: 22385.6406 - mean_squared_error: 22385.6406 - val_loss: 10179.1318 - val_mean_squared_error: 10179.1318
Epoch 140/20000
16/16 [==============================] - 0s 5ms/step - loss: 22379.3730 - mean_squared_error: 22379.3730 - val_loss: 10175.7432 - val_mean_squared_error: 10175.7432
Epoch 141/20000
16/16 [==============================] - 0s 5ms/step - loss: 22372.9766 - mean_squared_error: 2

16/16 [==============================] - 0s 5ms/step - loss: 22126.9258 - mean_squared_error: 22126.9258 - val_loss: 10042.5537 - val_mean_squared_error: 10042.5537
Epoch 182/20000
16/16 [==============================] - 0s 5ms/step - loss: 22120.6133 - mean_squared_error: 22120.6133 - val_loss: 10039.7139 - val_mean_squared_error: 10039.7139
Epoch 183/20000
16/16 [==============================] - 0s 5ms/step - loss: 22114.8691 - mean_squared_error: 22114.8691 - val_loss: 10036.6357 - val_mean_squared_error: 10036.6357
Epoch 184/20000
16/16 [==============================] - 0s 5ms/step - loss: 22108.9707 - mean_squared_error: 22108.9707 - val_loss: 10033.5498 - val_mean_squared_error: 10033.5498
Epoch 185/20000
16/16 [==============================] - 0s 5ms/step - loss: 22102.7559 - mean_squared_error: 22102.7559 - val_loss: 10030.0996 - val_mean_squared_error: 10030.0996
Epoch 186/20000
16/16 [==============================] - 0s 4ms/step - loss: 22096.5000 - mean_squared_error: 2

...

16/16 [==============================] - 0s 5ms/step - loss: 83.3766 - mean_squared_error: 83.3766 - val_loss: 191.3635 - val_mean_squared_error: 191.3635
Epoch 19797/20000
16/16 [==============================] - 0s 5ms/step - loss: 101.4359 - mean_squared_error: 101.4359 - val_loss: 165.4486 - val_mean_squared_error: 165.4486
Epoch 19798/20000
16/16 [==============================] - 0s 5ms/step - loss: 112.9130 - mean_squared_error: 112.9130 - val_loss: 133.2549 - val_mean_squared_error: 133.2549
Epoch 19799/20000
16/16 [==============================] - 0s 5ms/step - loss: 94.1115 - mean_squared_error: 94.1115 - val_loss: 130.4255 - val_mean_squared_error: 130.4255
Epoch 19800/20000
16/16 [==============================] - 0s 5ms/step - loss: 89.6762 - mean_squared_error: 89.6762 - val_loss: 163.5721 - val_mean_squared_error: 163.5721
Epoch 19801/20000
16/16 [==============================] - 0s 5ms/step - loss: 84.5052 - mean_squared_error: 84.5052 - val_loss: 154.1780 - val_mean_

16/16 [==============================] - 0s 5ms/step - loss: 90.4497 - mean_squared_error: 90.4497 - val_loss: 85.8335 - val_mean_squared_error: 85.8335
Epoch 19844/20000
16/16 [==============================] - 0s 5ms/step - loss: 84.4063 - mean_squared_error: 84.4063 - val_loss: 165.0598 - val_mean_squared_error: 165.0598
Epoch 19845/20000
16/16 [==============================] - 0s 5ms/step - loss: 103.1665 - mean_squared_error: 103.1665 - val_loss: 31.0888 - val_mean_squared_error: 31.0888
Epoch 19846/20000
16/16 [==============================] - 0s 5ms/step - loss: 132.5632 - mean_squared_error: 132.5632 - val_loss: 58.5462 - val_mean_squared_error: 58.5462
Epoch 19847/20000
16/16 [==============================] - 0s 5ms/step - loss: 77.1478 - mean_squared_error: 77.1478 - val_loss: 78.1168 - val_mean_squared_error: 78.1168
Epoch 19848/20000
16/16 [==============================] - 0s 5ms/step - loss: 83.6741 - mean_squared_error: 83.6741 - val_loss: 20.8988 - val_mean_squared_e

16/16 [==============================] - 0s 5ms/step - loss: 98.3999 - mean_squared_error: 98.3999 - val_loss: 44.2059 - val_mean_squared_error: 44.2059
Epoch 19891/20000
16/16 [==============================] - 0s 5ms/step - loss: 116.1963 - mean_squared_error: 116.1963 - val_loss: 94.6072 - val_mean_squared_error: 94.6072
Epoch 19892/20000
16/16 [==============================] - 0s 5ms/step - loss: 103.9534 - mean_squared_error: 103.9534 - val_loss: 194.2255 - val_mean_squared_error: 194.2255
Epoch 19893/20000
16/16 [==============================] - 0s 5ms/step - loss: 90.6619 - mean_squared_error: 90.6619 - val_loss: 94.3130 - val_mean_squared_error: 94.3130
Epoch 19894/20000
16/16 [==============================] - 0s 5ms/step - loss: 85.7094 - mean_squared_error: 85.7094 - val_loss: 36.1797 - val_mean_squared_error: 36.1797
Epoch 19895/20000
16/16 [==============================] - 0s 5ms/step - loss: 89.2932 - mean_squared_error: 89.2932 - val_loss: 131.7820 - val_mean_squared_

16/16 [==============================] - 0s 5ms/step - loss: 89.8296 - mean_squared_error: 89.8296 - val_loss: 102.5474 - val_mean_squared_error: 102.5474
Epoch 19938/20000
16/16 [==============================] - 0s 5ms/step - loss: 169.6682 - mean_squared_error: 169.6682 - val_loss: 35.2721 - val_mean_squared_error: 35.2721
Epoch 19939/20000
16/16 [==============================] - 0s 5ms/step - loss: 79.1236 - mean_squared_error: 79.1236 - val_loss: 78.3391 - val_mean_squared_error: 78.3391
Epoch 19940/20000
16/16 [==============================] - 0s 6ms/step - loss: 73.2023 - mean_squared_error: 73.2023 - val_loss: 198.8411 - val_mean_squared_error: 198.8411
Epoch 19941/20000
16/16 [==============================] - 0s 5ms/step - loss: 96.9803 - mean_squared_error: 96.9803 - val_loss: 151.3868 - val_mean_squared_error: 151.3868
Epoch 19942/20000
16/16 [==============================] - 0s 5ms/step - loss: 79.3815 - mean_squared_error: 79.3815 - val_loss: 27.6632 - val_mean_squared

16/16 [==============================] - 0s 5ms/step - loss: 97.9962 - mean_squared_error: 97.9962 - val_loss: 25.3375 - val_mean_squared_error: 25.3375
Epoch 19985/20000
16/16 [==============================] - 0s 5ms/step - loss: 66.0112 - mean_squared_error: 66.0112 - val_loss: 23.0421 - val_mean_squared_error: 23.0421
Epoch 19986/20000
16/16 [==============================] - 0s 5ms/step - loss: 70.1861 - mean_squared_error: 70.1861 - val_loss: 20.1871 - val_mean_squared_error: 20.1871
Epoch 19987/20000
16/16 [==============================] - 0s 5ms/step - loss: 76.2161 - mean_squared_error: 76.2161 - val_loss: 47.3443 - val_mean_squared_error: 47.3443
Epoch 19988/20000
16/16 [==============================] - 0s 5ms/step - loss: 88.1487 - mean_squared_error: 88.1487 - val_loss: 41.2185 - val_mean_squared_error: 41.2185
Epoch 19989/20000
16/16 [==============================] - 0s 5ms/step - loss: 98.1219 - mean_squared_error: 98.1219 - val_loss: 89.4860 - val_mean_squared_error: 

In [39]:
y_pred = np.squeeze(model.predict(X_test))
((y_pred-y_test)**2).mean()

12/12 [==============================] - 0s 942us/step


65.49775023647784

In [40]:
train_preds = np.squeeze(model.predict(X_train))
((train_preds-y_train)**2).mean()

16/16 [==============================] - 0s 920us/step


72.40347228446032